<a href="https://colab.research.google.com/github/Shalaka127/Nidra-Ai/blob/main/Final_dream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install openai
!pip install python-dotenv
!pip install pandas
!pip install matplotlib
!pip install seaborn



In [ ]:
!pip install --upgrade openai


In [ ]:
import gradio as gr
import openai
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
import openai
import os

In [ ]:


from google.colab import userdata
api_key= userdata.get('API_KEY')
#print(api_key)

import os
os.environ['OPENAI_API_KEY'] = api_key
os.getenv('OPENAI_API_KEY')



In [ ]:
from openai import OpenAI

client = OpenAI()

In [ ]:


# --- Helper Functions ---
def get_symbol_meanings(symbols):
    """Gets symbol meanings from OpenAI."""
    symbol_str = ", ".join(symbols)
    prompt = f"Provide a short cultural or symbolic meaning for each of the following symbols, separate each meaning with a semicolon (;). Only provide the cultural or symbolic meaning. Do not provide the symbol.: {symbol_str}"
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant providing cultural and symbolic meanings."},
                {"role": "user", "content": prompt}
            ]
        )
        meanings = response.choices[0].message.content.split(';')
        # Clean and return the meanings
        return [meaning.strip() for meaning in meanings]
    except Exception as e:
        print(f"Error getting symbol meanings from OpenAI: {e}")
        return ["No specific cultural meaning found."] * len(symbols)  # Default meaning

def analyze_dream(dream_text):
    """Analyzes dream and extracts parameters and provides an interpretation using OpenAI."""

    prompt = f"""You are a dream analyst. Analyze the following dream narrative and provide:

    1.  A Dream Interpretation: A detailed explanation of the dream's possible meanings, symbols, and overall message. Be thorough and insightful.
    2.  Extracted Parameters: Extract the following, providing each parameter on a new line, without any asterisks or markdown formatting:
        Dominant Emotion: <the single most prominent emotion in the dream>
        Recurring Symbols: <list of symbols separated by semicolons (;)>

    Format your response STRICTLY. Adhere to the exact formatting instructions above, without using any asterisks or markdown characters.
    Here's the dream:

    {dream_text}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a dream analyst."},
                {"role": "user", "content": prompt}
            ]
        )
        analysis = response.choices[0].message.content
        interpretation = extract_interpretation(analysis)  # Extract the interpretation
        dream_params = extract_parameters(analysis)  # Extract parameters

        # Prepare Outputs
        symbol_data = []
        if dream_params and "recurring_symbols" in dream_params:
            symbols = dream_params["recurring_symbols"]
            symbol_meanings_list = get_symbol_meanings(symbols)
            for i, symbol in enumerate(symbols):
                meaning = symbol_meanings_list[i] if i < len(symbol_meanings_list) else "Meaning not found"
                symbol_data.append({"Symbol": symbol, "Cultural Meaning": meaning})
        symbol_df = pd.DataFrame(symbol_data)

        emotion_plot = None # No plot
        if dream_params and "dominant_emotion" in dream_params and dream_params["dominant_emotion"]:
                dominant_emotion = dream_params["dominant_emotion"]
                plt.figure(figsize=(6, 4))
                plt.bar(dominant_emotion, 7, color='skyblue')
                plt.xlabel("Emotion")
                plt.ylabel("Intensity")
                plt.title("Dominant Dream Emotion")
                plt.tight_layout()
                emotion_plot = plt # set the emotion as the name of the plot
        return interpretation, symbol_df, emotion_plot # Now returning data as Dataframes and plots

    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None

def extract_interpretation(analysis):
    """Extracts the dream interpretation from the analysis text."""
    interpretation_pattern = r"A Dream Interpretation:\s*(.*)(?=(Dominant Emotion:)|(\Z))"
    match = re.search(interpretation_pattern, analysis, re.DOTALL)
    if match:
        interpretation = match.group(1).strip()
        # Remove asterisks from the interpretation
        interpretation = interpretation.replace("*", "")
        return interpretation
    else:
        return "No dream interpretation found in the analysis."

def extract_parameters(analysis):
    """Extracts the dream parameters from the analysis text."""
    dream_params = {
        "dominant_emotion": "",  # Only most prominent Emotion
        "recurring_symbols": []
    }

    # Use regular expressions to find the value for each parameter
    dominant_emotion_match = re.search(r"Dominant Emotion:\s*(.*)(?=(Recurring Symbols:)|(\Z))", analysis, re.DOTALL)
    symbols_match = re.search(r"Recurring Symbols:\s*(.*)", analysis, re.DOTALL)

    # Update the dream_params dictionary if a match is found
    if dominant_emotion_match:
        dream_params["dominant_emotion"] = dominant_emotion_match.group(1).strip()
    if symbols_match:
        dream_params["recurring_symbols"] = [s.strip() for s in symbols_match.group(1).split(';')]

    return dream_params


# --- Gradio Interface ---
with gr.Blocks() as iface:
    gr.Markdown("# Dream Analyzer")

    dream_text_input = gr.Textbox(label="Enter your dream here:", lines=5)
    analyze_button = gr.Button("Analyze Dream")

    interpretation_output = gr.Textbox(label="Dream Interpretation:")
    symbol_table_output = gr.DataFrame(label="Symbol Interpretations")
    emotion_plot_output = gr.Plot(label="Dominant Dream Emotion")

    analyze_button.click(
        analyze_dream,
        inputs=dream_text_input,
        outputs=[interpretation_output, symbol_table_output, emotion_plot_output]
    )

if __name__ == "__main__":
    iface.launch()